<a href="https://colab.research.google.com/github/sharmaa4/Twitter-Bot-for-Research-Paper-Summarization-using-Natural-Language-Processing/blob/main/Final_Submisision_of_Research_Paper_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -Uq fastbook
!rm -rf /content/pdfminer.six
!pip install pdfminer.six

import os
import pandas as pd
import numpy as np
import requests

import tweepy
import webbrowser
import time

from lxml import etree
from tqdm.auto import tqdm
from fastai.text.all import *
!git clone "https://github.com/pdfminer/pdfminer.six"
!pip install pdfminer.six
!pip install -Uq transformers
!pip install -r '/content/drive/MyDrive/Colab Notebooks/requirements.txt'

## XML PARSING TO CONVERT INPUT DATA IN XML FORMAT TO STRINGS AND APPENDING THEM INTO A CSV FILE

In [ ]:
root_path = '/content/drive/MyDrive/Colab Notebooks/scisummnet_release1.1__20190413/top1000_complete'

In [ ]:
pathlist = os.listdir(root_path)
# for folder in pathlist:
#   print(os.listdir(root_path+'/'+folder+'/summary'))

In [ ]:
parser = etree.XMLParser(remove_blank_text=True)

# papers
pap_text = []
summaries = []
for folder in tqdm(pathlist):
  tree = etree.parse(root_path+'/'+folder+'/Documents_xml/'+folder+'.xml', parser)
  paper = []
  for s in tree.iter():
    if s.text is not None: paper.append(s.text)

  with open(root_path+'/'+folder+'/summary/'+folder+'.gold.txt') as f:
    lines = f.read()

  pap_text.append(' '.join(paper))
  summaries.append(lines)

  0%|          | 0/1009 [00:00<?, ?it/s]

In [ ]:
scis_df = pd.DataFrame({'text':pap_text, 'summary':summaries})
type(scis_df['text'].iloc[0])

str

In [ ]:
scis_df.to_csv('./scisumm.csv', index=False)

In [ ]:
import numpy as np

# Manual train-test split
np.random.seed(10) # seed added 
msk = np.random.rand(len(scis_df)) < 0.8
msk
train_df = scis_df[msk]
test_df = scis_df[~msk]
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/scisumm_train.csv')
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/scisumm_test.csv')

## FINE-TUNING BART WITH HUGGINGFACE SCRIPT

In [ ]:
!git clone https://github.com/huggingface/transformers
os.chdir("/content/transformers")
!pwd
!pip install .

!python '/content/drive/MyDrive/Colab Notebooks/run_summarization.py' \
    --model_name_or_path facebook/bart-large-cnn \
    --do_train \
    --do_eval \
    --train_file '/content/drive/MyDrive/Colab Notebooks/scisumm_train.csv' \
    --validation_file '/content/drive/MyDrive/Colab Notebooks/scisumm_test.csv' \
    --text_column text \
    --summary_column summary \
    --source_prefix "summarize: " \
    --output_dir "/content/drive/MyDrive/Colab Notebooks/Summarization Output/" \
    --overwrite_output_dir \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --predict_with_generate

## MEASURING PERFORMANCE OF FINE-TUNED MODEL

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

PATH = "/content/drive/MyDrive/Colab Notebooks/Summarization Output/"
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained(PATH, local_files_only=True)
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(PATH, local_files_only=True)


In [ ]:
gen_summs = []
for paper in tqdm(test_df.text.values[:203]):
#for paper in tqdm(scis_df.text.values[:5]):

  ARTICLE_TO_SUMMARIZE = paper
  inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt', truncation=True)

  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=500, early_stopping=True)
  gen_summs.append([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

gen_df = pd.DataFrame({'generated_summary':gen_summs})
gen_df.head()

gen_df.generated_summary = gen_df.generated_summary.apply(lambda x: x[0])
gen_df.head()

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge2', 'rouge3'])
scores = []
#mod: sharmaa4 for i in range(500):
for i in range(203):
  
  scores.append(scorer.score(test_df.iloc[i].summary, gen_df.iloc[i].generated_summary))


r2r = []
r2f = []
r3f = []
#for i in range(250):
for i in range(203):
  r2r.append(scores[i]['rouge2'].recall)
  r2f.append(scores[i]['rouge2'].fmeasure)
  r3f.append(scores[i]['rouge3'].fmeasure)


print('rouge2 - recall: '+ str(np.mean(r2r)))
print('rouge2 - fmeasure: '+ str(np.mean(r2f)))
print('rouge3 - fmeasure: '+ str(np.mean(r3f)))

gen_df.to_csv('./nonpt_first500.csv', index=False)

## FUNCTION TO GENERATE SUMMARY 

In [ ]:
os.chdir("/content")

In [ ]:
!git clone "https://github.com/pdfminer/pdfminer.six"

fatal: destination path 'pdfminer.six' already exists and is not an empty directory.


In [ ]:
def gen_pdf_summary(url):

  print(url)
  !curl -o PDF {url}
  !python /content/pdfminer.six/tools/pdf2txt.py PDF > ./PDF_text

  with open("./PDF_text",'r') as file:
    text = file.read()
  
  inputs_1 = tokenizer(text[0:1024], max_length=1024, return_tensors='pt', truncation=True)
  inputs_2 = tokenizer(text[1025:2048], max_length=1024, return_tensors='pt', truncation=True)
  inputs_3 = tokenizer(text[2048:3072], max_length=1024, return_tensors='pt', truncation=True)
  inputs_4 = tokenizer(text[3072:], max_length=1024, return_tensors='pt', truncation=True)

  # Generate Summary
  summary_ids_1 = model.generate(inputs_1['input_ids'], num_beams=4, max_length=100, early_stopping=True)
  summary_1 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_1]

  summary_ids_2 = model.generate(inputs_2['input_ids'], num_beams=4, max_length=100, early_stopping=True)
  summary_2 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_2]

  summary_ids_3 = model.generate(inputs_3['input_ids'], num_beams=4, max_length=100, early_stopping=True)
  summary_3 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_3]

  summary_ids_4 = model.generate(inputs_4['input_ids'], num_beams=4, max_length=100, early_stopping=True)
  summary_4 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_4]
  
  return summary_1[0]+summary_2[0]+summary_3[0]+summary_4[0]





In [ ]:
#gen_pdf_summary("https://arxiv.org/pdf/1909.01716v3.pdf")

https://arxiv.org/pdf/1909.01716v3.pdf
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  614k  100  614k    0     0   423k      0  0:00:01  0:00:01 --:--:--  423k


'ScisummNet: A Large Annotated Corpus and Content-Impact Models\nFor Scientiﬁc Paper Summarization with Citation Networks\nRecent work in automatic summarization has achieved good results for news articles: single-Document Single-Document (SDS) summaries (Parveen, Ramsl, and Strube) and traditional citation-based summaries.\nOur large annotated corpus and hybrid methods provide a new framework for science-oriented paper summarization research.\nWe demonstrate the effectiveness of our corpus in producing large-scale manually-annotated summaries and the advantage of'

## TWITTER BOT

In [ ]:
# these are the necessary keys which we got from Twitter
##consumer_key = "Qm4Exmimzawm2WJ1wCIbBPDCK"
##consumer_secret = "jDW4rd4sqwWM0t0nkEUXoRQnXYfpQjnJt4vqaxVhtfIaUb9PN9"
##access_token = "1419176160127164417-hiXZsBeeONG7cUKDdkz28fvPlU5J6r"
##access_token_secret = "kiyMn8EoCM3FbAF0jNHCBA59TtblJndRKmICwFv0ATIKW"

consumer_key = "VXE0kwXAw2bUJ9dx5gnFNmLEG"
consumer_secret = "LiebjWsYOcOXHWFsTTcfyxmPf13vWEc7xsgmrkVMJ4uXupkdI4"
access_token = "1419176160127164417-C4b5obrAO2EpwFnWBKC6XtQ6SJWwrO"
access_token_secret = "6WvekVxHlnQ6GWKSq1APhVPq3GpQg9gAd0NDBpzfP5k5V"

In [ ]:
# as of now we need to get tweets from this user, so we pick his user ID
userID = "ak92501"

In [ ]:
# authorizing our twitter credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
# user_timeline extends tweets from particular user
tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = True,
                           tweet_mode = 'extended'
                           )

In [ ]:
# these are the recent 10 tweets from AK
def extract_pdf_link_from_tweet():

  for info in tweets[14:15]:
    link = ""
    print("ID: {}".format(info.id))
    print(info.created_at)
    print(info.full_text)
    print("\n")
    pdf_link = re.search("pdf:.*",info.full_text)
     

    if(pdf_link != None):
      pdf_link_list = pdf_link.string.split("\n")
      for i in pdf_link_list:
         if("pdf: " in i) :
            print("PDF Link: ",i)
            link_list = i.split(" ")
            link = link_list[1]
          

  return link      


#extract_pdf_link_from_tweet()      

ID: 1420856997583413262
2021-07-29 21:21:21
RT @abidlabs: By now, you've probably heard about Hugging Face Spaces, a *free* way to host @Gradio demos, like this cool depth perception…




''

In [ ]:
import requests
def unshorten_url(url=""):
  
  session = requests.Session()  # so connections are recycled
  resp = session.head(url, allow_redirects=True)
  
  return resp.url

In [ ]:
# Posting on Twitter
def tweet_summary(tweet="",user_id=""):
  
  tweet_list = tweet.split('\n')
  tweet_list
  count = 0
  my_id="@Summarizer19 "
  for i in tweet_list[:-1] :
      count = count+1
      if(count == 1):
        head = "Requested By "+"@"+user_id+" "
        print(i)
        try:
          original_tweet = api.update_status(status=head+i)
        except tweepy.TweepError:
          print("Could not tweet due to some issue")
          break
      else:
        print(my_id+i)
        if(count == 2):
          try: 
            reply_tweet = api.update_status(status=my_id+i, 
                                 in_reply_to_status_id=original_tweet.id, 
                                 auto_populate_reply_metadata=False)
          except tweepy.TweepError:
            print("Could not tweet due to some issue")
            break
        else:
          try:
            reply_tweet = api.update_status(status=my_id+i, 
                                 in_reply_to_status_id=reply_tweet.id, 
                                 auto_populate_reply_metadata=False)
          except tweepy.TweepError:
            print("Could not tweet due to some issue")
            break
        
  count = 0      

#tweet_summary(gen_pdf_summary(unshorten_url(extract_pdf_link_from_tweet())),user_id="ak92501")

In [ ]:
api.list_direct_messages(1)

[DirectMessage(_api=<tweepy.api.API object at 0x7f7afc6b3190>, _json={'type': 'message_create', 'id': '1421023655350259719', 'created_timestamp': '1627633415435', 'message_create': {'target': {'recipient_id': '1419176160127164417'}, 'sender_id': '1321213011911004161', 'message_data': {'text': 'Hi', 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}}}}, type='message_create', id='1421023655350259719', created_timestamp='1627633415435', message_create={'target': {'recipient_id': '1419176160127164417'}, 'sender_id': '1321213011911004161', 'message_data': {'text': 'Hi', 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}}})]

In [ ]:
def lookup_for_dms():
  global latest_dm_timestamp
  
  

  messages = api.list_direct_messages(1)

  
    

  output_link = ""
  output_user_id = ""

  for message in messages:
     if(int(message.created_timestamp) <= int(latest_dm_timestamp)):
       break

     if(len(message.message_create['message_data']['entities']['urls']) != 0):
       print(message.message_create['message_data']['entities']['urls'])
       #link = str(message.message_create['message_data']['entities']['urls'])
       link = str(message.message_create['message_data']['entities']['urls'])
       list_link= link.split(",")
       #print(list_link[1])
       list_expanded_link = list_link[1].split(": ")
       if("arxiv.org" in list_expanded_link[1]):
         expanded_arxiv_link = list_expanded_link[1]
         print(expanded_arxiv_link)
         output_link = expanded_arxiv_link
       else:
         print("Not an arxiv link\n")

     else:
       print("DM does not have a link")


     if(len(message.message_create['message_data']['entities']['user_mentions']) != 0):
    
       user_mentions = str(message.message_create['message_data']['entities']['user_mentions'])

       user_mentions_list = user_mentions.split(",")
       screen_name = user_mentions_list[0].split(": ")
       user_name = screen_name[1]
    
       print("User_id: ",user_name)

       output_user_id = user_name
  
     else:
       print("No username provided")

  if(int(message.created_timestamp) > int(latest_dm_timestamp)):
    output = []  
    output.append(output_link) 
    output.append(output_user_id)
  
  else:
    output = []  
    output.append(None) 
    output.append(None)

  latest_dm_timestamp = message.created_timestamp

  
  

  print(message)

  

  return output

#latest_dm_timestamp = 0
#requests = lookup_for_dms() 
#if(requests[0] != None):
#  print("LINK:",requests[0])
#  print("LAST DM TIMESTAMP",latest_dm_timestamp)
#else:
#  print("No Link")
#  print("LAST DM TIMESTAMP",latest_dm_timestamp)
#tweet_summary(gen_pdf_summary(requests[0]),user_id=requests[1][1:-1])  
  

In [ ]:
import time
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

#latest_dm_timestamp = 0

while True:
  print("Hello\n")
  requests = lookup_for_dms()
  if(requests[0] != None ):
    if(requests[0] != ""):
      print("LINK:",requests[0])
      tweet_summary(gen_pdf_summary(requests[0]),user_id=requests[1][1:-1])
  else:
    print("No Link in the latest DM or Summary has been already posted for the current DM")
    

  
  logger.info("Waiting...")
  time.sleep(60)